In [1]:
import pandas as pd
data = pd.read_csv('C:/Users/bam/Downloads/Compressed/data-2024/data.csv')

In [3]:
print(data.columns)


Index(['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId',
       'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId',
       'ProductCategory', 'ChannelId', 'Amount', 'Value',
       'TransactionStartTime', 'PricingStrategy', 'FraudResult'],
      dtype='object')


In [4]:
# Create aggregate features for each customer
aggregate_features = data.groupby('CustomerId').agg(
    total_transaction_amount=('Amount', 'sum'),
    average_transaction_amount=('Amount', 'mean'),
    transaction_count=('TransactionId', 'count'), 
    std_dev_transaction_amount=('Amount', 'std')  
).reset_index()
